In [54]:
import sys
import re
import pandas as pd
import json
import os
import dotenv

from pathlib import Path
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from openai import OpenAI

dotenv.load_dotenv()

True

In [55]:
df = pd.read_csv("SampleData_ComSci.csv")
df

,Source,Content,Page,Total_pages
0,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หลักสูตรวิทยาศาสตรบัณฑิต \nสาขาวิชาวิทยาการคอม...,1,205
1,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,รายละเอียดของหลักสูตร \nหลักสูตรวิทยาศาสตรบัณฑ...,2,205
2,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,1.4.2 ประเภทของหลักสูตร \n \n R \nหลักสูตรปริ...,3,205
3,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,1.5 อาชีพที่สามารถประกอบไดหลังสำเร็จการศึกษา ...,4,205
4,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวดที่ 2 คุณสมบัติผูเข้าศึกษา \n2.1 การรับเ...,5,205
...,...,...,...,...
177,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,ข้อ ๕6 ให้นับรวมเวลาในระหว่างที่ถูกลงโทษให้พัก...,179,205
178,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,ตามหลักเกณฑ์การวัดผลของหลักสูตรที่จะย้ายเข้า เ...,180,205
179,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวด ๑๖ \nการได้รับปริญญาเกียรตินิยม \n-------...,181,205
180,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวด ๑๗ \nการเสนอชื่อและการอนุมัติอนุปริญญาหรื...,182,205


In [56]:
typhoon_prompt = PromptTemplate(
    input_variables=["context"],
    template="""
    คุณเป็นผู้ช่วย AI ที่มีความสามารถในการวิเคราะห์และสรุปข้อมูลจากข้อความที่ให้มา 
    โดยเฉพาะอย่างยิ่งเมื่อมีข้อมูลเกี่ยวกับหลักสูตรการศึกษา 
    เช่น ชื่อหลักสูตร รหัสหลักสูตร ชื่อปริญญา สาขาวิชา วิชาเอก และรูปแบบของหลักสูตร

    {context}

    จากข้อความช่วยหา Key ของหน้านี้หน่อยว่ามีอะไรบ้าง และแสดงผลรับออกมาในรูปแบบของjson แต่ในกรณีที่มี R หรือ o ให้เอามาแค่ที่เป็น R พอ โดยไม่มีข้อความเพิ่มเติมใด ๆ
    """
)

In [57]:
#typhoon_token = os.getenv("TYPHOON_API_KEY")

# Initialize the Typhoon client
client = OpenAI(
    api_key=os.getenv("TYPHOON_API_KEY"),
    base_url='https://api.opentyphoon.ai/v1'
)

# Define a function to generate a response using the LLM
def generate_response(context, typhoon_prompt):
    system_prompt=typhoon_prompt.format(context=context)
    chat_completion = client.chat.completions.create(
        model="typhoon-v2-70b-instruct",
        messages=[{"role": "user", "content": system_prompt}],
        max_tokens=2048,
        temperature=0.7,
    )
    return chat_completion.choices[0].message.content

In [58]:
import pandas as pd

def generate_response_for_all_rows(df, typhoon_prompt):
    responses_list = []
    
    for index, row in df.iterrows():
        context = row['Content']
        
        system_prompt = typhoon_prompt.format(context=context)
        chat_completion = client.chat.completions.create(
            model="typhoon-v2-70b-instruct",
            messages=[{"role": "user", "content": system_prompt}],
            max_tokens=2048,
            temperature=0.7,
        )
        response = chat_completion.choices[0].message.content
        responses_list.append(response)

    df['Response'] = responses_list
    
    return df


In [62]:
df_test = generate_response_for_all_rows(df, typhoon_prompt)

In [63]:
df_test

,Source,Content,Page,Total_pages,Response
0,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หลักสูตรวิทยาศาสตรบัณฑิต \nสาขาวิชาวิทยาการคอม...,1,205,"{\n ""ชื่อหลักสูตร"": ""หลักสูตรวิทยาศาสตรบัณฑิต..."
1,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,รายละเอียดของหลักสูตร \nหลักสูตรวิทยาศาสตรบัณฑ...,2,205,"{\n ""ชื่อหลักสูตร"": ""หลักสูตรวิทยาศาสตรบัณฑิต..."
2,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,1.4.2 ประเภทของหลักสูตร \n \n R \nหลักสูตรปริ...,3,205,"```json\n{\n ""ประเภทของหลักสูตร"": ""หลักสูตรปร..."
3,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,1.5 อาชีพที่สามารถประกอบไดหลังสำเร็จการศึกษา ...,4,205,"```json\n{\n ""อาชีพที่สามารถประกอบได้หลังสำเร..."
4,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวดที่ 2 คุณสมบัติผูเข้าศึกษา \n2.1 การรับเ...,5,205,"{\n ""หมวดที่ 2"": {\n ""คุณสมบัติผูเข้าศึกษ..."
...,...,...,...,...,...
177,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,ข้อ ๕6 ให้นับรวมเวลาในระหว่างที่ถูกลงโทษให้พัก...,179,205,"{\n ""R"": [""ข้อ ๕6"", ""ข้อ ๕7"", ""ข้อ ๕8"", ""ข้อ ..."
178,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,ตามหลักเกณฑ์การวัดผลของหลักสูตรที่จะย้ายเข้า เ...,180,205,"{\n ""key"": [\n ""นักศึกษา"",\n ""หลักสูตร""..."
179,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวด ๑๖ \nการได้รับปริญญาเกียรตินิยม \n-------...,181,205,"{\n ""Key"": [\n ""การได้รับปริญญาเกียรตินิยม..."
180,วิทยาการคอมพิวเตอร์ หลักสูตร 2566-V4.pdf,หมวด ๑๗ \nการเสนอชื่อและการอนุมัติอนุปริญญาหรื...,182,205,"{\n ""หมวด"": [\n ""17"",\n ""18""\n ],\n ""..."


In [64]:
df_test.to_csv("ComSci_Response.csv", index=False ,encoding='utf-8-sig')